# STATS507 Assignment 2
### **Kailan Xu**  
### *September 22, 2021*

***
## Question 0 - Code review  warmup [10 points]                   

### Code Snippet

In [ ]:
sample_list = [(1, 3, 5), (0, 1, 2), (1, 9, 8)]
op = []
for m in range(len(sample_list)):  
    li = [sample_list[m]]   
        for n in range(len(sample_list)):
            if (sample_list[m][0] == sample_list[n][0] and
                    sample_list[m][3] != sample_list[n][3]):
                li.append(sample_list[n])
        op.append(sorted(li, key=lambda dd: dd[3], reverse=True)[0])
res = list(set(op))

a. Concisely describe what task the code above accomplishes.

- From the sample list, the code would pick out the tuples with the biggest 3rd element group by the 1st element of tuples. 

b. Write a short code review that offers 3-5 (no more) concrete suggestions to make the snippent more efficient, literate (easier to read), or “pythonic”.

1. The indentation of line 5 is 4 spaces more.
1. The index 3 is out of size of tuples, it should be 2.
1. The name of variables in the snippets is lack of readability. 
1. The loop in loop ($O(n^2)$) could be replaced bya loop with a dictionary ($O(n)$)


***
## Question 1 - List of Tuples [20 points]

Write a function that uses NumPy and a list comprehension to generate a random list of `n` k-tuples containing integers ranging from `low` to `high`. Choose an appropriate name for your function, and reasonable default values for k, `low`, and `high`.

Use `assert` to test that your function returns a list of tuples.

In [1]:
# libraries: -----------------------------------------------------------------
import pandas as pd
import numpy as np
import time 
from scipy.stats import norm
from timeit import Timer
from collections import defaultdict

In [2]:
# function: ------------------------------------------------------------------
def generate_lst_of_tuples(n, k=3, low=1, high=10):
    """
    Generate a random list of n k-tuples.
    
    Parameter
    ---------
    n: int, the length of the list
    k: int, the length of the tuples
    low: int, the lower bound of integers of tuples
    high: int, the upper bound of integers of tuples
    
    Returns
    ---------
    A list of n k-tuples.
    """
    
    rd_array = np.random.choice(range(low, high), n*k)
    array_reshape = rd_array.reshape(n,k)
    re = list(tuple([x for x in y]) for y in array_reshape)
    
    return re
# 79: ------------------------------------------------------------------------

In [3]:
# test: ----------------------------------------------------------------------
re = generate_lst_of_tuples(2)
assert isinstance(re,list), 'Output is not a list'
assert isinstance(re[0],tuple), 'The elements are not tuples.'
print('The function returns a list of tuples')

# 79: ------------------------------------------------------------------------

The function returns a list of tuples


***
### Question 2 - Refactor the Snippet [40 points]


In this question, you will write functions to accomplish the goal you concisely described in part “a” of the warm up.

1. Encapsulate the code snippet from the warmup into a function that parameterizes the role of 0 and 3 and is otherwise unchanged. Choose appropriate names for these parameters.

In [4]:
# function: ------------------------------------------------------------------
def rank_tuples_groupby_idxa_orderby_idxb(sample_list, a=0, b=2):
    """
    From the sample list, pick out the tuples with the biggest b th element 
    group by the a th element of tuples. Ordinary Version
    
    Parameter
    ---------
    sample_list: a list of tuples
    a: an int
    b: an int
    
    Returns
    ---------
    A list of tuples.
    """
    op = []
    for m in range(len(sample_list)):  
        li = [sample_list[m]]   
        for n in range(len(sample_list)):
            if (sample_list[m][a] == sample_list[n][a] and
                    sample_list[m][b] != sample_list[n][b]):
                li.append(sample_list[n])
        op.append(sorted(li, key=lambda dd: dd[b], reverse=True)[0])
    res = list(set(op))
    
    return res
# 79: ------------------------------------------------------------------------

2. Write an improved version of the function form part a that implements the suggestions from the code review you wrote in part b of the warmup.

In [5]:
# function: ------------------------------------------------------------------
def rank_tuples_groupby_idxa_orderby_idxb_v2(sample_list, a=0, b=2):
    """
    From the sample list, pick out the tuples with the biggest bth element 
    group by the ath element of tuples. Polished Version of Loop in Loop.
    
    Parameter
    ---------
    sample_list: a list of tuples
    a: an int
    b: an int
    
    Returns
    ---------
    A list of tuples.
    """
    op = []
    for m in range(len(sample_list)): 
        M = sample_list[m]
        li = [M]   
        for n in range(len(sample_list)):
            N = sample_list[n]
            if (M[a] == N[a] and M[b] != N[b]):
                li.append(N)
        op.append(sorted(li, key=lambda dd: dd[b], reverse=True)[0])
    res = list(set(op))
    
    return res
# 79: ------------------------------------------------------------------------

3. Write a function from scratch to accomplish the same task as the previous two parts. Your solution should traverse the input list of tuples no more than twice. Hint: consider using a dictionary or a default dictionary in your solution.

In [6]:
# function: ------------------------------------------------------------------
def rank_tuples_groupby_idxa_orderby_idxb_oneloop(sample_list, a=0, b=2):
    """
    From the sample list, pick out the tuples with the biggest bth element 
    group by the ath element of tuples. Loop once with a dictionary.
    
    Parameter
    ---------
    sample_list: a list of tuples
    a: an int
    b: an int
    
    Returns
    ---------
    A list of tuples.
    """
    op = {}
    for m in range(len(sample_list)):  
        A = sample_list[m][a]
        B = sample_list[m][b]
        if str(A) not in op.keys():
            op[str(A)] = sample_list[m]
        else:
            if op[str(A)][b] < B:
                op[str(A)] = sample_list[m]
    
    return list(op.values())
# 79: ------------------------------------------------------------------------

4. Use the function you wrote in question 1 to generate a list of tuples as input(s), run and summarize a small Monte Carlo study comparing the execution times of the three functions above (a-c)

In [8]:
# timing with list of tuples input: ------------------------------------------
sample_lst = generate_lst_of_tuples(100)
function_lst = [rank_tuples_groupby_idxa_orderby_idxb, 
                rank_tuples_groupby_idxa_orderby_idxb_v2, 
                rank_tuples_groupby_idxa_orderby_idxb_oneloop]
time_nda = defaultdict(list)

for f in function_lst:
    t = Timer('f(sample_lst)', globals={'f': f, 'sample_lst': sample_lst})
    tm = t.repeat(repeat=10, number=100)
    time_nda['Function'].append(f.__name__)
    time_nda['min, s'].append(np.min(tm))
    time_nda['median, s'].append(np.median(tm))
    time_nda['mean, s'].append(np.mean(tm))

time_nda = pd.DataFrame(time_nda)
for c, d in zip(time_nda.columns, time_nda.dtypes):
    if d == np.dtype('float64'):
        time_nda[c] = time_nda[c].map(lambda x: '%5.3f' % x)
time_nda
# 79: ------------------------------------------------------------------------

,Function,"min, s","median, s","mean, s"
0,rank_tuples_groupby_idxa_orderby_idxb,0.119,0.125,0.128
1,rank_tuples_groupby_idxa_orderby_idxb_v2,0.100,0.107,0.129
2,rank_tuples_groupby_idxa_orderby_idxb_oneloop,0.010,0.010,0.010


**Summary:**  
- In this Monte Carlo simulation, I set 10 groups of parellel experiments, and each group consisting of 100 times of function executions.By computing the minimum/median/mean time of each 100 execution times of each function, we can compare functions' running efficiency. 
- The simple Monte Carlo study implies that for every 100 times execution,the minimum time consumed for function `rank_tuples_groupby_idxa_orderby_idxb` is 0.121s, for function `rank_tuples_groupby_idxa_orderby_idxb_v2` is 0.098s and for function `rank_tuples_groupby_idxa_orderby_idxb_oneloop` is 0.01s which is a square root level of execution time of former 2 functions.

***
### Question 3 - [30 points]

1. Use Python and Pandas to read and append the demographic datasets keeping only columns containing the unique ids (SEQN), age (RIDAGEYR), race and ethnicity (RIDRETH3), education (DMDEDUC2), and marital status (DMDMARTL), along with the following variables related to the survey weighting: (RIDSTATR, SDMVPSU, SDMVSTRA, WTMEC2YR, WTINT2YR). Add an additional column identifying to which cohort each case belongs. Rename the columns with literate variable names using all lower case and convert each column to an appropriate type. Finally, save the resulting data frame to a serialized “round-trip” format of your choosing (e.g. pickle, feather, or parquet).

In [8]:
# Read data: -----------------------------------------------------------------
Demo1 = pd.read_sas('DEMO_G.XPT'); Demo1['cohort']='2011-2012'
Demo2 = pd.read_sas('DEMO_H.XPT'); Demo2['cohort']='2013-2014'
Demo3 = pd.read_sas('DEMO_I.XPT'); Demo3['cohort']='2015-2016'
Demo4 = pd.read_sas('DEMO_J.XPT'); Demo4['cohort']='2017-2018'

# Clean data: ----------------------------------------------------------------
Demo = Demo1.append(Demo2).append(Demo3).append(Demo4).reset_index(drop=True)
Demo = Demo[['SEQN','RIDAGEYR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR',
             'SDMVPSU', 'SDMVSTRA', 'WTMEC2YR', 'WTINT2YR','cohort']]
Demo = Demo.fillna(-1) ## Transfer NaN to -1

# Convert data type: ---------------------------------------------------------
for i in ['SEQN','RIDAGEYR','RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR',
             'SDMVPSU', 'SDMVSTRA']:
    Demo[i] = Demo[i].astype(int, errors='ignore')

# Remame the columns: --------------------------------------------------------
Demo = Demo.rename({'SEQN':'id','RIDAGEYR':'age','RIDRETH3':
                    'race and ethnicity', 'DMDEDUC2':'education',
                    'DMDMARTL':'marital status'},axis=1)

# Save the results: ----------------------------------------------------------
Demo.to_pickle('./Demo_pickle.pkl')

# 79: ------------------------------------------------------------------------

2. Repeat part a for the oral health and dentition data (OHXDEN_*.XPT) retaining the following variables: SEQN, OHDDESTS, tooth counts (OHXxxTC), and coronal cavities (OHXxxCTC).

In [11]:
# Read data: ------------------------------------------------------------------
OHX1 = pd.read_sas('OHXDEN_G.XPT'); OHX1['cohort']='2011-2012'
OHX2 = pd.read_sas('OHXDEN_H.XPT'); OHX2['cohort']='2013-2014'
OHX3 = pd.read_sas('OHXDEN_I.XPT'); OHX3['cohort']='2015-2016'
OHX4 = pd.read_sas('OHXDEN_J.XPT'); OHX4['cohort']='2017-2018'

# Clean data: ----------------------------------------------------------------
OHX = OHX1.append(OHX2).append(OHX3).append(OHX4).reset_index(drop=True)
OHX = pd.concat([OHX.iloc[:,:64].drop(['OHDEXSTS','OHXIMP'],axis=1),
                                                 OHX['cohort']],axis=1)
OHX = OHX.fillna(-1) ## Transfer NaN to -1

# Convert data type: ---------------------------------------------------------
for i in range(33):
    OHX[str(OHX.columns[i])] = OHX[str(OHX.columns[i])].astype(
        int, errors='ignore')

# Remame the columns: --------------------------------------------------------
OHX = OHX.rename({'SEQN':'id','OHDDESTS':'dentition status code'},axis=1)

# Save the results: ----------------------------------------------------------
OHX.to_pickle('./OHX_pickle.pkl')

# 79: ------------------------------------------------------------------------

3. In your notebook, report the number of cases there are in the two datasets above.

In [17]:
print('The number of cases in the demographic datasets(2011-2018) is:',Demo['id'].nunique())
print('The number of cases in the  oral health and dentition datasets(2011-2018) is:',OHX['id'].nunique())

The number of cases in the demographic datasets(2011-2018) is: 39156
The number of cases in the  oral health and dentition datasets(2011-2018) is: 35909


- The number of cases in the demographic datasets(2011-2018) is: 39156
- The number of cases in the  oral health and dentition datasets(2011-2018) is: 35909